In [6]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format,concat,substring,lit,lpad,to_timestamp,when, avg,year
import  pandas as pd
import os

In [7]:
spark = SparkSession.builder \
    .appName("Combine CSV Files and Calculate Average") \
    .getOrCreate()

In [15]:
result_df = spark.createDataFrame([], schema="year INT, winddir DOUBLE, sky DOUBLE, visdist DOUBLE, airtemp DOUBLE, dewpointtemp DOUBLE, atmpress DOUBLE")


In [19]:
for current_year in range(1976, 2025):
    # Read CSV file for the current year
    file_path = f"./dataPreprocessed/{current_year}.csv"
    df = spark.read.option("header", "true").csv(file_path)

    # Parse the 'date' column to extract the year
    df = df.withColumn("year", year("date"))

    # Group the DataFrame by year and calculate the average of each column
    avg_df = df.groupBy("year").agg(
       
      
        avg("winddir").alias("winddir"),
        avg("sky").alias("sky"),
        avg("visdist").alias("visdist"),
        avg("airtemp").alias("airtemp"),
        avg("dewpointtemp").alias("dewpointtemp"),
        avg("atmpress").alias("atmpress")
    )
    result_df = result_df.union(avg_df)
result_df.show()

DataFrame[summary: string, year: string, winddir: string, sky: string, visdist: string, airtemp: string, dewpointtemp: string, atmpress: string]